In [ ]:
from collections import defaultdict

import networkx as nx
import pandas as pd
import pyomo.environ as pe

# What are indexed sets?

In [ ]:
N = 20
p = 0.20
graph = nx.erdos_renyi_graph(N, p, seed=18)
nx.draw_networkx(graph)

In [ ]:
adjacent = defaultdict(set)
for (i, j) in graph.edges:
    adjacent[i].add(j)
    adjacent[j].add(i)

In [ ]:
for node in graph.nodes:
    print(node, adjacent[node])

# Why do we need them?

Mostly for the sake of expeditiously building model implementations. In this example, the indexed sets are a "cheap storage" or "sparse matrix" version of the adjacency matrix. In very many applications, the full adjacency matrix is chock full of zeroes and not practical to store.

In [ ]:
df = pd.Series({(i, j): 1 for i in adjacent for j in adjacent[i]}).to_frame().reset_index()
df.columns = ['i', 'j', 'adjacency']
pd.crosstab(df['i'], df['j'])

# Can they be implemented in Pyomo?

In [ ]:
# we did this all before
model = pe.ConcreteModel()
nodes = set(graph.nodes)
edges = set(graph.edges)
model.nodes = pe.Set(initialize=nodes)
model.edges = pe.Set(within=model.nodes*model.nodes, initialize=edges)

# but this is new
model.adjacent = pe.Param(model.nodes, within=pe.Any, default=set(), initialize=adjacent)

In [ ]:
dict(model.adjacent)

# A Non-Graph Example

A power grid is comprised of many buses (i.e., nodes), and each bus is associated with some number of generator units. For example:

In [ ]:
B = {'A', 'B', 'C', 'D', 'E', 'F'}
G = {1, 2, 3, 4, 5, 6, 7}
G_of_b = {
    'A': set(),
    'B': {1},
    'C': {2, 3},
    'D': {4},
    'E': {5, 6},
    'F': {7},
}

In [ ]:
dict(G_of_b)

In [ ]:
df = pd.Series({(b, g): 1 for b in B for g in G_of_b[b]}).to_frame().reset_index()
df.columns = ['b', 'g', 'adjacency']
pd.crosstab(df['b'], df['g'])

Suppose we want to build a model where (active) power generation is a variable at every (bus, generator) pair. We set up index sets for the buses and generators separately as expected. To allow the model to grasp which generators belong to which buses, we set up a pair of objects that have similar function and compare.

- `model.BxG`: A subset of the Cartesian product of the bus and generator sets.
- `model.G_of_b`: A partition of the set of generators into sets that are indexed in the bus set.

In [ ]:
model = pe.ConcreteModel()
model.B = pe.Set(initialize=B)
model.G = pe.Set(initialize=G)
model.BxG = pe.Set(within=model.B*model.G, initialize={(b, g) for b in B for g in G_of_b[b]})
model.G_of_b = pe.Param(model.B, within=pe.Any, default=set(), initialize=G_of_b)

We need `model.BxG`, a proper Pyomo set, to construct the variables.

In [ ]:
model.p = pe.Var(model.BxG, domain=pe.NonNegativeReals)

Now how do we build an expression of all power generated? We can do this at least two different ways.

1. Using only `model.BxG`.
2. Using `model.G_of_b` (`pe.Param`) with `model.B` (`pe.Set`).

Both methods perform similarly. 

In [ ]:
expr = sum(model.p[b, g] for (b, g) in model.BxG)
print(expr)

In [ ]:
expr = sum(model.p[b, g] for b in model.B for g in model.G_of_b[b])
print(expr)

How about power generated just at bus A? Again, we can do this at least two different ways.

1. Using `model.BxG`.
2. Using `model.G_of_b`.

This time, the latter method is probably preferable for the same reason. The latter is constructive while the former method is deconstructive.

In [ ]:
expr = sum(model.p[b, g] for (b, g) in model.BxG if b == 'A')
print(expr)

In [ ]:
expr = sum(model.p['A', g] for g in model.G_of_b['A'])
print(expr)